# Interface between 2D materials: Boron Nitride and Graphene

## Introduction

This notebook demonstrates the creation of an interface between two 2D materials, Boron Nitride (BN) and Graphene and shifting the film along the y-axis to create multiple (7) stacking configurations.

Following the manuscript:
> **Jeil Jung, Ashley M. DaSilva, Allan H. MacDonald & Shaffique Adam**
> **Origin of the band gap in graphene on hexagonal boron nitride**
> Nature Communications volume 6, Article number: 6308 (2015)
> [DOI: 10.1038/ncomms7308](https://doi.org/10.1038/ncomms7308)


Relicating the materials with profile give in Figure 7. a (top row):

<img src='https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/interfaces/interface_2d_2d_graphene_boron_nitride/0-figure-from-manuscript.webp' width='700'/>


## 1. Prepare the Environment
### 1.1. Set up the notebook 

Set the following flags to control the notebook behavior 

In [ ]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False

FILM_INDEX = 1  # Index in the list of materials, to access as materials[FILM_INDEX]
FILM_MILLER_INDICES = (0, 0, 1)
FILM_THICKNESS = 1  # in atomic layers
FILM_VACUUM = 0.0  # in angstroms
FILM_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
FILM_USE_ORTHOGONAL_C = True

SUBSTRATE_INDEX = 0
SUBSTRATE_MILLER_INDICES = (0, 0, 1)
SUBSTRATE_THICKNESS = 1  # in atomic layers
SUBSTRATE_VACUUM = 0.0  # in angstroms
SUBSTRATE_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
SUBSTRATE_USE_ORTHOGONAL_C = True

# Maximum area for the superlattice search algorithm
MAX_AREA = 350  # in Angstrom^2
# Set the termination pair indices
INTERFACE_DISTANCE = 3.4  # in Angstrom
INTERFACE_VACUUM = 20.0  # in Angstrom

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples|create_interface_with_min_strain_zsl.ipynb")

### 1.3. Get input materials and assign `substrate` and `film`
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

film = Material.create(Materials.get_by_name_first_match("Graphene"))
substrate = Material.create(Materials.get_by_name_and_categories("BN", "2D"))

### 1.4. Preview Substrate and Film

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

## 2. Configure slabs and select termination pair

### 2.1. Create Substrate and Layer Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [ ]:
from mat3ra.made.tools.build.slab.helpers import create_slab

film_slab = create_slab(
    crystal=film,
    miller_indices=FILM_MILLER_INDICES,
    number_of_layers=FILM_THICKNESS,  # in atomic layers
    vacuum=FILM_VACUUM,  # in angstroms
    xy_supercell_matrix=FILM_XY_SUPERCELL_MATRIX,
    use_orthogonal_c=FILM_USE_ORTHOGONAL_C
)

substrate_slab = create_slab(
    crystal=substrate,
    miller_indices=SUBSTRATE_MILLER_INDICES,
    number_of_layers=SUBSTRATE_THICKNESS,  # in atomic layers
    vacuum=SUBSTRATE_VACUUM,  # in angstroms
    xy_supercell_matrix=SUBSTRATE_XY_SUPERCELL_MATRIX,
    use_orthogonal_c=SUBSTRATE_USE_ORTHOGONAL_C
)

# Visualize the created slabs
visualize([{"material": film_slab, "title": "Film Slab"}, {"material": substrate_slab, "title": "Substrate Slab"}],
          repetitions=[3, 3, 1], rotation="-90x")


## 3. Create interfaces

### 3.1. Initialize the Interface Configuration

In [ ]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration

interface_configuration = InterfaceConfiguration(
    film_configuration=film_slab_configuration,
    substrate_configuration=substrate_slab_configuration,
    film_termination=film_termination,
    substrate_termination=substrate_termination,
    distance=INTERFACE_DISTANCE,
    vacuum=INTERFACE_VACUUM
)

### 3.2. Set Strain Matching Algorithm Parameters (Optional)
The search algorithm for supercells matching can be tuned by setting its parameters directly, otherwise the default values are used.

In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingParameters

zsl_strain_matching_parameters = ZSLStrainMatchingParameters(
    max_area=MAX_AREA
)

### 3.3. Generate interfaces with strain matcher
Interfaces are sorted by size and strain.

In [ ]:
from mat3ra.made.tools.build.interface import ZSLStrainMatchingInterfaceBuilder, \
    ZSLStrainMatchingInterfaceBuilderParameters

matched_interfaces_builder = ZSLStrainMatchingInterfaceBuilder(
    build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
        strain_matching_parameters=zsl_strain_matching_parameters))

interfaces_sorted_by_size_and_strain = matched_interfaces_builder.get_materials(configuration=interface_configuration)

### 3.4. Plot interfaces by size and strain


In [ ]:
from utils.plot import plot_strain_vs_atoms

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_atoms(interfaces_sorted_by_size_and_strain, PLOT_SETTINGS)

### 3.5. Select the interface

Select the index for the interface with the lowest strain and the smallest number of atoms.

In [ ]:
# select the first interface with the lowest strain and the smallest number of atoms
interfaces_slice_range_or_index = 0
selected_interface = interfaces_sorted_by_size_and_strain[interfaces_slice_range_or_index]

### 4.1 Preview the selected material

In [ ]:
visualize(selected_interface, repetitions=[3, 3, 1])
visualize(selected_interface, repetitions=[3, 3, 1], rotation="-90x")

### 4.2. Shift film along y-axis
Shifting with the step of a/sqrt(3)/2 angstroms, where a is the lattice constant of the interface.

In [ ]:
import numpy as np
from mat3ra.made.tools.modify import interface_displace_part

a = selected_interface.lattice.a
shifted_interfaces = []
for n in range(2, 9):
    shifted_interface = interface_displace_part(
        interface=selected_interface,
        displacement=[0, n * a / np.sqrt(3) / 2, 0],
        use_cartesian_coordinates=True)
    shifted_interfaces.append(shifted_interface)

### 4.3. Preview the shifted materials

In [ ]:
visualize(shifted_interfaces, repetitions=[3, 3, 1])

## 5. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials(shifted_interfaces)
for idx, interface in enumerate(shifted_interfaces):
    download_content_to_file(interface, f"interface_{idx}.json")